MTG Card Identifier  

In [1]:
import requests
import pandas as pd
import sys

print("Python:", sys.version.split()[0])
print("requests:", requests.__version__)
print("pandas:", pd.__version__)


Python: 3.13.3
requests: 2.32.3
pandas: 2.3.0


In [2]:
def fetch_card_data(card_name: str) -> dict:
    """
    Fetch card data from Scryfall API by exact name.
    Returns a dict with Name, Set, Rarity, Oracle Text, Image URL, Price USD.
    """
    url = f"https://api.scryfall.com/cards/named?exact={card_name}"
    resp = requests.get(url)
    resp.raise_for_status()
    data = resp.json()
    return {
        "Name": data.get("name"),
        "Set": data.get("set_name"),
        "Rarity": data.get("rarity"),
        "Oracle Text": data.get("oracle_text"),
        "Image URL": data.get("image_uris", {}).get("normal"),
        "Price USD": data.get("prices", {}).get("usd")
    }


In [6]:
sample = ["Lightning Bolt"]
card_info = fetch_card_data(sample)
df = pd.DataFrame([card_info])
display(df)


,Name,Set,Rarity,Oracle Text,Image URL,Price USD
0,Lightning Bolt,Ravnica: Clue Edition,uncommon,Lightning Bolt deals 3 damage to any target.,https://cards.scryfall.io/normal/front/7/7/77c...,0.86
